Assignment 5 \
Part 3: DT-Wage Regression \
Submitted by Group 5 \

Groupmembers:

1. Sajan Kumer Sarker (2111131642) (sajan.sarker@northsouth.edu)
2. Rafsan Jani Chowdhury (2011424642) (rafsan.chowdhury@northsouth.edu)
3. Rosely Mohammad (2014219642) (rosely.mohammad@northsouth.edu)

Task: \
1) Support vector regression (SVR)


**For Regression problem: Mean-squared error**

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import math
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.preprocessing import LabelEncoder, OneHotEncoder, StandardScaler
from sklearn.svm import SVC, SVR
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from collections import defaultdict

In [3]:
df = pd.read_csv('/content/drive/MyDrive/CSE445-Assignment/dataset/DT-Wage.csv')
print(df.shape)
print(df.head())

(3000, 11)
   year  age            maritl      race        education              region  \
0  2006   18  1. Never Married  1. White     1. < HS Grad  2. Middle Atlantic   
1  2004   24  1. Never Married  1. White  4. College Grad  2. Middle Atlantic   
2  2003   45        2. Married  1. White  3. Some College  2. Middle Atlantic   
3  2003   43        2. Married  3. Asian  4. College Grad  2. Middle Atlantic   
4  2005   50       4. Divorced  1. White       2. HS Grad  2. Middle Atlantic   

         jobclass          health health_ins   logwage        wage  
0   1. Industrial       1. <=Good      2. No  4.318063   75.043154  
1  2. Information  2. >=Very Good      2. No  4.255273   70.476020  
2   1. Industrial       1. <=Good     1. Yes  4.875061  130.982177  
3  2. Information  2. >=Very Good     1. Yes  5.041393  154.685293  
4  2. Information       1. <=Good     1. Yes  4.318063   75.043154  


In [4]:
print('Number of missing values: ')
print(df.isnull().sum())

Number of missing values: 
year          0
age           0
maritl        0
race          0
education     0
region        0
jobclass      0
health        0
health_ins    0
logwage       0
wage          0
dtype: int64


In [5]:
df = pd.get_dummies(df, columns=['maritl', 'race', 'education', 'region', 'jobclass', 'health', 'health_ins'])
df.columns = df.columns.str.replace('[', '_').str.replace(']', '_').str.replace('<', '_')
print(df.shape)
print(df.head())

(3000, 25)
   year  age   logwage        wage  maritl_1. Never Married  \
0  2006   18  4.318063   75.043154                     True   
1  2004   24  4.255273   70.476020                     True   
2  2003   45  4.875061  130.982177                    False   
3  2003   43  5.041393  154.685293                    False   
4  2005   50  4.318063   75.043154                    False   

   maritl_2. Married  maritl_3. Widowed  maritl_4. Divorced  \
0              False              False               False   
1              False              False               False   
2               True              False               False   
3               True              False               False   
4              False              False                True   

   maritl_5. Separated  race_1. White  ...  education_3. Some College  \
0                False           True  ...                      False   
1                False           True  ...                      False   
2           

In [6]:
X = df.drop(columns=['wage'])
y = df['wage']

X_train, X_temp, y_train, y_temp = train_test_split(X, y, test_size=0.3, random_state=42)
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.5, random_state=42)

print(f'Train Data Shape (X, y): {X_train.shape, y_train.shape}')
print(f'Validation Data Shape (X, y): {X_val.shape, y_val.shape}')
print(f'Test Data Shape (X, y): {X_test.shape, y_test.shape}')

Train Data Shape (X, y): ((2100, 24), (2100,))
Validation Data Shape (X, y): ((450, 24), (450,))
Test Data Shape (X, y): ((450, 24), (450,))


In [8]:
# mse calculation
def mse(y_actual, y_pred):
    error = 0.0

    for actual, predicted in zip(y_actual, y_pred):
        error += (actual - predicted) ** 2
    mse = error / len(y_actual)
    return mse

In [9]:
# SVM base model
svm_base = SVR()
svm_base.fit(X_train, y_train)

y_val_pred = svm_base.predict(X_val)
print('Validation Set MSE:', mse(y_val, y_val_pred))

Validation Set MSE: 1680.9641962289247


In [10]:
param_grid = {
    'C': [0.1, 1, 10, 100],
    'epsilon': [0.01, 0.1, 0.2, 0.5],
    'kernel': ['linear', 'rbf', 'poly']
}

svr = SVR()
svm_grid_search = GridSearchCV(estimator=svr, param_grid=param_grid, cv=5, scoring='neg_mean_squared_error', n_jobs=-1, verbose=1)
svm_grid_search.fit(X_train, y_train)

best_params = svm_grid_search.best_params_
print("Best Parameters:", best_params)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
Best Parameters: {'C': 1, 'epsilon': 0.5, 'kernel': 'linear'}


In [11]:
svm_final = SVR(**best_params)
svm_final.fit(X_train, y_train)

y_val_pred = svm_final.predict(X_val)
print('Validation Set MSE:', mse(y_val, y_val_pred))

Validation Set MSE: 375.3116751678494


In [14]:
# now retrain the model on the combined training and validation set with the best hyperparameters
X_train_val = np.concatenate((X_train, X_val))
y_train_val = np.concatenate((y_train, y_val))

svm_final.fit(X_train_val, y_train_val)

y_test_pred = svm_final.predict(X_test)
print('Test Set MSE:', mse(y_test, y_test_pred))

Test Set MSE: 358.61853766502287
